In [1]:
import numpy as np
import pandas as pd
import dotenv
import os
import requests

dotenv.load_dotenv()

True

In [26]:
# get api key
key = os.getenv('STOCK_DATA_API_KEY')
url = os.getenv('STOCK_PRICE_DATA_API_URL')
url_adjusted = os.getenv('ADJUSTED_INTRADAY_URL')
url_unadjusted = os.getenv('UNADJUSTED_INTRADAY_URL')

In [22]:
# test getting data for some tickers
# free tier only allows 1 ticker for an intraday request
params = {
    "api_token": key,
    "symbols": ['MSFT'],
    "extended_hours": True,
    "key_by_ticker": True
}
response = requests.get(url, params)
response.json()

{'meta': {'requested': 1, 'returned': 1},
 'data': [{'ticker': 'MSFT',
   'name': 'Microsoft Corporation',
   'exchange_short': None,
   'exchange_long': None,
   'mic_code': 'IEXG',
   'currency': 'USD',
   'price': 507.29,
   'day_high': 510.7,
   'day_low': 502.48,
   'day_open': 504.3,
   '52_week_high': None,
   '52_week_low': None,
   'market_cap': None,
   'previous_close_price': 504.22,
   'previous_close_price_time': '2025-08-21T15:59:57.000000',
   'day_change': 0.61,
   'volume': 702446,
   'is_extended_hours_price': False,
   'last_trade_time': '2025-08-22T16:00:00.000000'}]}

In [50]:
# intraday unadjusted (adjusted is not free)
params = {
    "api_token": key,
    "symbols": ["AAPL"],
    "sort": "desc",
    "date_from": "2024-06-25",
    "date_to": "2024-06-30",
    "key_by_date": True,
}
response = requests.get(url_unadjusted, params)

In [51]:
# look at data
response_dict = dict(response.json())
response_dict.keys()
metadata = response_dict['meta']
data = response_dict['data']
metadata

{'date_from': '2025-07-25', 'date_to': '2025-08-01', 'max_period_days': 7}

In [52]:
# lets look at the data structure
df = pd.DataFrame(data)
print(df.columns)
print(df.head(5))

Index(['date', 'ticker', 'data'], dtype='object')
                       date ticker  \
0  2025-08-01T15:59:00.000Z   AAPL   
1  2025-08-01T15:58:00.000Z   AAPL   
2  2025-08-01T15:57:00.000Z   AAPL   
3  2025-08-01T15:56:00.000Z   AAPL   
4  2025-08-01T15:55:00.000Z   AAPL   

                                                data  
0  {'open': 202.38, 'high': 202.45, 'low': 202.19...  
1  {'open': 202.52, 'high': 202.54, 'low': 202.34...  
2  {'open': 202.62, 'high': 202.64, 'low': 202.37...  
3  {'open': 202.61, 'high': 202.71, 'low': 202.54...  
4  {'open': 202.77, 'high': 203.01, 'low': 202.44...  


In [53]:
# need to destructure the dataframe
df['data'][0].keys()
df_flat = pd.concat([df.drop(columns=["data"]), pd.json_normalize(df["data"])], axis=1)
df_flat.head(5)

,date,ticker,open,high,low,close,volume,is_extended_hours
0,2025-08-01T15:59:00.000Z,AAPL,202.38,202.45,202.19,202.37,33846,False
1,2025-08-01T15:58:00.000Z,AAPL,202.52,202.54,202.34,202.39,14766,False
2,2025-08-01T15:57:00.000Z,AAPL,202.62,202.64,202.37,202.54,13493,False
3,2025-08-01T15:56:00.000Z,AAPL,202.61,202.71,202.54,202.59,9979,False
4,2025-08-01T15:55:00.000Z,AAPL,202.77,203.01,202.44,202.59,16783,False


In [2]:
from data_handler import DataHandler

In [ ]:
api = DataHandler()
df = api.get_data(["VOO"])

In [ ]:
# look at data 
print(df.head()) 
print(len(df)) 
print("Earliest record: " + df['date'][0])
print("Latest record: " + df['date'][1488])
print("Number of records: " + len(df))

                       date ticker    open    high     low   close  volume  \
0  2025-08-18T09:30:00.000Z    VOO  590.98  591.23  590.95  591.16    2140   
1  2025-08-18T09:31:00.000Z    VOO  591.34  591.34  591.10  591.10     400   
2  2025-08-18T09:32:00.000Z    VOO  591.14  591.14  591.14  591.14       1   
3  2025-08-18T09:33:00.000Z    VOO  591.20  591.30  591.20  591.30     122   
4  2025-08-18T09:34:00.000Z    VOO  591.26  591.48  591.26  591.48      16   

   is_extended_hours  
0              False  
1              False  
2              False  
3              False  
4              False  
1489
2025-08-18T09:30:00.000Z
2025-08-22T15:59:00.000Z


In [12]:
# save to csv to save on api calls
api.save_to_csv("VOO_8_18_25_to_8_22_25", df)

True